In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [16]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6])


In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1794, -0.1662, -0.0730,  0.5147, -0.3418, -0.0835,  0.0406,  0.0181,
         -0.1141,  0.3478,  0.0687,  0.7444, -0.0773, -0.0399, -0.1921,  0.0994,
         -0.2226, -0.3393,  0.1132,  0.1274],
        [-0.2119,  0.1445, -0.2253,  0.4126, -0.1584,  0.2990,  0.0710,  0.2912,
         -0.3681,  0.3356,  0.2458,  0.7822,  0.1373, -0.1562, -0.1074,  0.1715,
          0.0721, -0.2849, -0.0832,  0.4183],
        [-0.4316, -0.3009, -0.0411,  0.1180, -0.5135,  0.2465, -0.0591,  0.0184,
         -0.2570,  0.4435,  0.2466,  0.8820, -0.0034, -0.0856, -0.2737,  0.1111,
          0.2533, -0.3465, -0.2028,  0.1040]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.5147, 0.0000, 0.0000, 0.0406, 0.0181, 0.0000,
         0.3478, 0.0687, 0.7444, 0.0000, 0.0000, 0.0000, 0.0994, 0.0000, 0.0000,
         0.1132, 0.1274],
        [0.0000, 0.1445, 0.0000, 0.4126, 0.0000, 0.2990, 0.0710, 0.2912, 0.0000,
         0.3356, 0.2458, 0.7822, 0.1373, 0.0000, 0.00

In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [24]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0037, -0.0294,  0.0003,  ...,  0.0339, -0.0311, -0.0095],
        [-0.0228,  0.0105, -0.0163,  ...,  0.0052,  0.0142, -0.0187]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0147, 0.0003], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0115, -0.0275,  0.0438,  ...,  0.0252,  0.0024,  0.0133],
        [ 0.0009,  0.0323, -0.0117,  ...,  0.0421, -0.0137, -0.0008]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Si